In [ ]:
import os, httpx, json
from pprint import pprint

API_KEY = os.getenv("THESPORTSDB_KEY", "3")  # free demo key
BASE_URL = f"https://www.thesportsdb.com/api/v1/json/{API_KEY}"
print("Using BASE_URL:", BASE_URL)

def GET(path, params=None):
    url = f"{BASE_URL}/{path.lstrip('/')}"
    r = httpx.get(url, params=params or {}, timeout=20)
    r.raise_for_status()
    data = r.json()
    # Normalize empty payloads like {"teams": None}
    if isinstance(data, dict) and data and all(v is None for v in data.values()):
        return {}
    return data

In [ ]:
sports = GET("all_sports.php").get("sports") or []
print("Total sports:", len(sports))
pprint(sports[0])  # first full record

In [ ]:
leagues = GET("all_leagues.php").get("leagues") or []
print("Total leagues:", len(leagues))
pprint(leagues[2])  # see all keys of one league

In [ ]:
LEAGUE_ID = "4330"  # English Premier League
teams = GET("lookup_all_teams.php", {"id": LEAGUE_ID}).get("teams") or []
print("Teams in league", LEAGUE_ID, ":", len(teams))
pprint(teams[0])  # one team record with all fields

In [ ]:
seasons = GET("search_all_seasons.php", {"id": LEAGUE_ID}).get("seasons") or []
print("Seasons for league", LEAGUE_ID, ":", len(seasons))
pprint(seasons)

In [ ]:
past_matches = GET("eventspastleague.php", {"id": LEAGUE_ID}).get("events") or []
next_matches = GET("eventsnextleague.php", {"id": LEAGUE_ID}).get("events") or []
print("Past matches:", len(past_matches), "| Next matches:", len(next_matches))
pprint(past_matches[0])  # see all fields for one match

In [ ]:
last_team_matches = GET("eventslast.php", {"id": TEAM_ID}).get("results") or []
print("Last matches for team:", len(last_team_matches))
pprint(last_team_matches[0])

In [ ]:
EVENT_ID = past_matches[0]["idEvent"]  # take first past match
detail = GET("lookupevent.php", {"id": EVENT_ID}).get("events") or []
timeline = GET("lookuptimeline.php", {"id": EVENT_ID}).get("timeline") or []
stats = GET("lookupeventstats.php", {"id": EVENT_ID}).get("eventstats") or []
lineup = GET("lookuplineup.php", {"id": EVENT_ID}).get("lineup") or []

print("Event detail keys:", list(detail[0].keys()))
print("\nTimeline rows:", len(timeline))
pprint(timeline[:3])
print("\nStats rows:", len(stats))
pprint(stats[:3])
print("\nLineup rows:", len(lineup))
pprint(lineup[:1])

In [ ]:
VENUE_ID = "16163"  # Emirates Stadium
venue = GET("lookupvenue.php", {"id": VENUE_ID}).get("venues") or []
print("Venue fields:", list(venue[0].keys()))
pprint(venue[0])

In [ ]:
import httpx

API_KEY = "3"  # free demo key
BASE_URL = f"https://www.thesportsdb.com/api/v1/json/{API_KEY}"

endpoints = [
    "all_sports.php",
    "all_leagues.php",
    "search_all_leagues.php",
    "lookupleague.php",
    "lookup_all_teams.php",
    "searchteams.php",
    "lookupteam.php",
    "lookup_all_players.php",
    "searchplayers.php",
    "lookupplayer.php",
    "eventspastleague.php",
    "eventsnextleague.php",
    "eventslast.php",
    "eventsnext.php",
    "eventsseason.php",
    "eventsday.php",
    "lookupevent.php",
    "lookuptimeline.php",
    "lookupeventstats.php",
    "lookuplineup.php",
    "lookupvenue.php",
]

for ep in endpoints:
    url = f"{BASE_URL}/{ep}"
    try:
        r = httpx.get(url, timeout=10)
        if r.status_code == 200:
            print(f"✅ {ep}")
        else:
            print(f"⚠️ {ep} -> {r.status_code}")
    except Exception as e:
        print(f"❌ {ep} -> {e}")

In [ ]:
# Cell 2 — choose a match (event)
# OPTION A: paste a known event id
EVENT_ID = "2267081"  # <- replace with any event id you want

# OPTION B: quickly grab one from a league's recent/next fixtures
# (Uncomment to auto-pick the first from Premier League)
# league_id = "4328"
# past = GET("eventspastleague.php", {"id": league_id}).get("events") or []
# nxt  = GET("eventsnextleague.php", {"id": league_id}).get("events") or []
# EVENT_ID = (past or nxt)[0]["idEvent"]
# print("Auto-picked EVENT_ID:", EVENT_ID)

EVENT_ID

In [ ]:
# Base config
BASE = "http://127.0.0.1:8000/collect"   # change if your server is elsewhere

import requests, json, time
from typing import Dict, Any, Optional, List

def call(intent: str, args: Dict[str, Any] | None = None) -> Dict[str, Any]:
    payload = {"intent": intent, "args": args or {}}
    r = requests.post(BASE, json=payload, timeout=30)
    try:
        data = r.json()
    except Exception as e:
        raise RuntimeError(f"Non-JSON response (status={r.status_code}): {r.text}") from e
    return data

def assert_ok(resp: Dict[str, Any], msg: str = ""):
    if not resp.get("ok", False):
        pretty = json.dumps(resp, indent=2)
        raise AssertionError(f"Expected ok=True. {msg}\nResponse:\n{pretty}")

def show(resp: Dict[str, Any], keys: Optional[List[str]] = None, maxlen: int = 3):
    """Pretty-print a small slice of the response for quick human inspection."""
    print(json.dumps({
        "ok": resp.get("ok"),
        "intent": resp.get("intent"),
        "args_resolved": resp.get("args_resolved"),
        "data_preview": {
            k: (resp["data"].get(k)[:maxlen] if isinstance(resp["data"].get(k), list) else resp["data"].get(k))
            for k in (keys or list((resp.get("data") or {}).keys()))
        }
    }, indent=2))

In [ ]:
# Fetch leagues
leagues_resp = call("leagues.list", {})
assert_ok(leagues_resp, "leagues.list failed")
show(leagues_resp, keys=["leagues"], maxlen=5)

# Pick English Premier League if present, else take the first
leagues = leagues_resp["data"].get("leagues") or []
assert leagues, "No leagues returned"

epl = next((l for l in leagues if (l.get("strLeague") or "").lower() == "english premier league"), leagues[0])
LEAGUE_ID = epl["idLeague"]
LEAGUE_NAME = epl["strLeague"]
print("Chosen League:", LEAGUE_NAME, LEAGUE_ID)

In [ ]:
seasons_resp = call("seasons.list", {"leagueId": LEAGUE_ID})
assert_ok(seasons_resp, "seasons.list failed")
show(seasons_resp, keys=["seasons"], maxlen=10)

seasons = seasons_resp["data"].get("seasons") or []
assert seasons, "No seasons returned for league"
SEASON = seasons[0]["strSeason"]
print("Chosen Season:", SEASON)

In [ ]:
teams_resp = call("teams.list", {"leagueId": LEAGUE_ID})
assert_ok(teams_resp, "teams.list failed")
show(teams_resp, keys=["teams"], maxlen=10)

teams = teams_resp["data"].get("teams") or []
assert teams, "No teams returned for league"
TEAM = teams[0]
TEAM_ID = TEAM["idTeam"]
TEAM_NAME = TEAM["strTeam"]
print("Chosen Team:", TEAM_NAME, TEAM_ID)

In [ ]:
team_detail_resp = call("team.get", {"teamId": TEAM_ID})
assert_ok(team_detail_resp, "team.get failed")
show(team_detail_resp, keys=["team"])

In [ ]:
players_resp = call("players.list", {"teamId": TEAM_ID})
assert_ok(players_resp, "players.list failed")
show(players_resp, keys=["players"], maxlen=10)

players = players_resp["data"].get("players") or []
if players:
    PLAYER_ID = players[0]["idPlayer"]
    PLAYER_NAME = players[0]["strPlayer"]
    print("Chosen Player:", PLAYER_NAME, PLAYER_ID)
else:
    PLAYER_ID = None
    PLAYER_NAME = None
    print("No players found for this team (can happen on some datasets).")

In [ ]:
if PLAYER_ID:
    player_detail_resp = call("player.get", {"playerId": PLAYER_ID})
    assert_ok(player_detail_resp, "player.get failed")
    show(player_detail_resp, keys=["player"])
else:
    print("Skipping player.get — no player chosen.")

In [ ]:
events_resp = call("events.list", {"leagueId": LEAGUE_ID, "season": SEASON})
assert_ok(events_resp, "events.list (league+season) failed")
show(events_resp, keys=["events"], maxlen=10)

events = events_resp["data"].get("events") or []
if not events:
    print("No events returned for this league+season (try a different season).")
EVENT = events[0] if events else None
EVENT_ID = EVENT["idEvent"] if EVENT else None
print("Chosen Event:", EVENT_ID)

In [ ]:
if EVENT_ID:
    event_detail_resp = call("event.get", {"eventId": EVENT_ID, "expand": ["timeline", "stats", "lineup"]})
    assert_ok(event_detail_resp, "event.get failed")
    show(event_detail_resp, keys=["event", "timeline", "stats", "lineup"], maxlen=5)
else:
    print("Skipping event.get — no event chosen.")

In [ ]:
from datetime import date
today_str = date.today().isoformat()

events_by_day_resp = call("events.list", {"date": today_str})
# Could be ok=false if API has no events for today; handle gracefully
if events_by_day_resp.get("ok"):
    show(events_by_day_resp, keys=["events"], maxlen=10)
    print("Events-by-day count:", len(events_by_day_resp["data"].get("events") or []))
else:
    print("No events for today or error:", json.dumps(events_by_day_resp, indent=2))

In [ ]:
# LAST events for team
last_resp = call("events.list", {"teamId": TEAM_ID, "kind": "last"})
assert_ok(last_resp, "events.list (team last) failed")
show(last_resp, keys=["events"], maxlen=5)

# NEXT events for team
next_resp = call("events.list", {"teamId": TEAM_ID, "kind": "next"})
assert_ok(next_resp, "events.list (team next) failed")
show(next_resp, keys=["events"], maxlen=5)

In [ ]:
def pass_fail(intent: str, args: Dict[str, Any] | None = None) -> bool:
    try:
        resp = call(intent, args or {})
        assert_ok(resp)
        return True
    except Exception as e:
        print(f"[FAIL] {intent} {args} -> {e}")
        return False

results = {
    "leagues.list": pass_fail("leagues.list"),
    "seasons.list": pass_fail("seasons.list", {"leagueId": LEAGUE_ID}),
    "teams.list":   pass_fail("teams.list", {"leagueId": LEAGUE_ID}),
    "team.get":     pass_fail("team.get", {"teamId": TEAM_ID}),
    "players.list": pass_fail("players.list", {"teamId": TEAM_ID}),
}

if PLAYER_ID:
    results["player.get"] = pass_fail("player.get", {"playerId": PLAYER_ID})

if EVENT_ID:
    results["events.list (league+season)"] = pass_fail("events.list", {"leagueId": LEAGUE_ID, "season": SEASON})
    results["event.get"] = pass_fail("event.get", {"eventId": EVENT_ID, "expand": ["timeline"]})

print("\nSMOKE SUMMARY")
for k, v in results.items():
    print(f"{'PASS' if v else 'FAIL'} - {k}")

In [ ]:
import requests, time, re, csv, json
from typing import Dict, Any, List, Optional

BASE = "http://127.0.0.1:8000/collect"  # your FastAPI agent

def call(intent: str, args: Dict[str, Any] | None = None, timeout=60) -> Dict[str, Any]:
    r = requests.post(BASE, json={"intent": intent, "args": args or {}}, timeout=timeout)
    try:
        data = r.json()
    except Exception:
        raise RuntimeError(f"Non-JSON from agent (status {r.status_code}): {r.text[:500]}")
    return data

def assert_ok(resp: Dict[str, Any], ctx: str = "") -> Dict[str, Any]:
    if not resp.get("ok"):
        raise RuntimeError(f"Agent error {ctx}: {json.dumps(resp, indent=2)[:1000]}")
    return resp

# URL detectors
VIDEO_PAT = re.compile(r"(youtu\.be|youtube\.com|vimeo\.com|dai\.ly|dailymotion\.com|\.m3u8(\?|$)|\.mp4(\?|$)|\.mov(\?|$)|\.webm(\?|$))", re.I)

def pick_video_fields(obj: dict) -> List[str]:
    """Collect any string fields that look like video URLs."""
    out = []
    for k, v in (obj or {}).items():
        if isinstance(v, str) and "http" in v and VIDEO_PAT.search(v):
            out.append(v.strip())
    # de-dup, keep order
    seen, uniq = set(), []
    for u in out:
        if u not in seen:
            uniq.append(u); seen.add(u)
    return uniq

def write_csv(path: str, rows: List[dict], fieldnames: List[str]):
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        for r in rows:
            w.writerow({k: r.get(k, "") for k in fieldnames})

print("Agent:", BASE)

In [ ]:
import requests, time, re, json
from typing import Dict, Any, List, Optional

BASE = "http://127.0.0.1:8000/collect"  # your FastAPI agent

VIDEO_PAT = re.compile(r"(youtu\.be|youtube\.com|vimeo\.com|dai\.ly|dailymotion\.com|\.m3u8(\?|$)|\.mp4(\?|$)|\.mov(\?|$)|\.webm(\?|$))", re.I)

def pick_video_fields(obj: dict) -> List[str]:
    out = []
    for k, v in (obj or {}).items():
        if isinstance(v, str) and "http" in v and VIDEO_PAT.search(v):
            out.append(v.strip())
    # de-dup preserving order
    seen, uniq = set(), []
    for u in out:
        if u not in seen:
            uniq.append(u); seen.add(u)
    return uniq

def agent_call(intent: str, args: Dict[str, Any] | None = None, timeout=60) -> Dict[str, Any]:
    r = requests.post(BASE, json={"intent": intent, "args": args or {}}, timeout=timeout)
    try:
        return r.json()
    except Exception:
        raise RuntimeError(f"Non-JSON from agent (status {r.status_code}): {r.text[:500]}")

def call_ok(intent: str, args: Dict[str, Any] | None = None, ctx: str = "", max_retries: int = 5, base_sleep: float = 0.6):
    """Call the agent with exponential backoff if we see rate-limit-like failures."""
    attempt = 0
    while True:
        resp = agent_call(intent, args)
        if resp.get("ok", False):
            return resp
        # Retry on likely rate-limit or transient network issues
        msg = json.dumps(resp.get("error") or {}, ensure_ascii=False)
        if "429" in msg or "HTTP failed" in msg or "INTERNAL" in msg:
            if attempt >= max_retries:
                raise RuntimeError(f"Agent error (exhausted retries) {ctx} -> {msg}")
            sleep_s = base_sleep * (2 ** attempt)
            # jitter
            sleep_s += 0.1 * attempt
            time.sleep(sleep_s)
            attempt += 1
            continue
        # Non-retriable agent error
        raise RuntimeError(f"Agent error {ctx} -> {msg}")

print("Agent:", BASE)

In [ ]:
# Soft limits to avoid 429s
MAX_LEAGUES = 12          # scan first N leagues
SLEEP_BETWEEN = 0.2       # gentle throttle between requests

leagues_resp = call_ok("leagues.list", {}, "leagues.list")
LEAGUES = leagues_resp["data"].get("leagues") or []
print("Total soccer leagues available:", len(LEAGUES))
LEAGUES = LEAGUES[:MAX_LEAGUES]
print("Scanning leagues:", len(LEAGUES))

for idx, L in enumerate(LEAGUES, 1):
    lid = L.get("idLeague"); lname = L.get("strLeague")
    if not lid:
        continue

    # Try detail, but it's optional (we print base links even if detail fails retries)
    detail = None
    try:
        detail = call_ok("league.get", {"leagueId": lid}, f"league.get {lid}")
        league_obj = detail["data"].get("league") or {}
    except Exception as e:
        league_obj = {}
        print(f"[league.get WARN] {lid} {lname}: {e}")

    vids = pick_video_fields({**L, **league_obj})
    if vids:
        print(f"\n[LEAGUE] {lname} ({lid}) — {len(vids)} link(s)")
        for v in vids:
            print("  •", v)

    if idx % 4 == 0:
        print(f"[leagues] processed {idx}/{len(LEAGUES)}")
    time.sleep(SLEEP_BETWEEN)

In [ ]:
MAX_TEAMS_PER_LEAGUE = 8   # limit teams per league
SLEEP_BETWEEN = 0.2

for li, L in enumerate(LEAGUES, 1):
    lid = L.get("idLeague"); lname = L.get("strLeague")
    if not lid:
        continue

    teams_resp = call_ok("teams.list", {"leagueId": lid}, f"teams.list {lid}")
    teams = teams_resp["data"].get("teams") or []
    teams = teams[:MAX_TEAMS_PER_LEAGUE]
    print(f"\n[LEAGUE {lname}] teams to scan: {len(teams)}")

    for T in teams:
        tid = T.get("idTeam"); tname = T.get("strTeam")
        if not tid:
            continue

        # Try detail, but fallback to list object if rate-limited
        team_obj = {}
        try:
            tdetail = call_ok("team.get", {"teamId": tid}, f"team.get {tid}")
            team_obj = tdetail["data"].get("team") or {}
        except Exception as e:
            print(f"[team.get WARN] {tname} ({tid}): {e}")

        vids = pick_video_fields({**T, **team_obj})
        if vids:
            print(f"  [TEAM] {tname} ({tid}) — {len(vids)} link(s)")
            for v in vids:
                print("    •", v)

        time.sleep(SLEEP_BETWEEN)

    if li % 3 == 0:
        print(f"[teams] leagues processed {li}/{len(LEAGUES)}")
    time.sleep(SLEEP_BETWEEN)

In [ ]:
# --- Self-contained EVENTS scan via your agent ---

import requests, time, re, json
from typing import Dict, Any, List

BASE = "http://127.0.0.1:8000/collect"  # your FastAPI agent

VIDEO_PAT = re.compile(r"(youtu\.be|youtube\.com|vimeo\.com|dai\.ly|dailymotion\.com|\.m3u8(\?|$)|\.mp4(\?|$)|\.mov(\?|$)|\.webm(\?|$))", re.I)

def pick_video_fields(obj: dict) -> List[str]:
    out = []
    for k, v in (obj or {}).items():
        if isinstance(v, str) and "http" in v and VIDEO_PAT.search(v):
            out.append(v.strip())
    # de-dup preserving order
    seen, uniq = set(), []
    for u in out:
        if u not in seen:
            uniq.append(u); seen.add(u)
    return uniq

def agent_call(intent: str, args: Dict[str, Any] | None = None, timeout=60) -> Dict[str, Any]:
    r = requests.post(BASE, json={"intent": intent, "args": args or {}}, timeout=timeout)
    try:
        return r.json()
    except Exception:
        raise RuntimeError(f"Non-JSON from agent (status {r.status_code}): {r.text[:500]}")

def call_ok(intent: str, args: Dict[str, Any] | None = None, ctx: str = "", max_retries: int = 5, base_sleep: float = 0.6):
    """Call agent with exponential backoff for transient/429 errors."""
    attempt = 0
    while True:
        resp = agent_call(intent, args)
        if resp.get("ok", False):
            return resp
        msg = json.dumps(resp.get("error") or {}, ensure_ascii=False)
        # Retry on likely transient issues
        if "429" in msg or "HTTP failed" in msg or "INTERNAL" in msg:
            if attempt >= max_retries:
                raise RuntimeError(f"Agent error (exhausted retries) {ctx} -> {msg}")
            sleep_s = base_sleep * (2 ** attempt) + 0.1 * attempt
            time.sleep(sleep_s)
            attempt += 1
            continue
        # Non-retriable
        raise RuntimeError(f"Agent error {ctx} -> {msg}")

# Fetch leagues first (so LEAGUES is defined in this cell)
leagues_resp = call_ok("leagues.list", {}, "leagues.list")
LEAGUES: List[dict] = leagues_resp["data"].get("leagues") or []
print("Total soccer leagues available:", len(LEAGUES))

# --- CONFIG: tweak limits to avoid rate-limit ---
SEASONS_LIMIT = 1            # scan last N seasons per league
MAX_LEAGUES = 8              # scan only the first N leagues
MAX_EVENTS_PER_SEASON = 15   # cap events per season
SLEEP_BETWEEN = 0.25         # throttle between calls

LEAGUES = LEAGUES[:MAX_LEAGUES]
print("Scanning leagues:", len(LEAGUES))

def seasons_for_league(league_id: str) -> List[str]:
    sresp = call_ok("seasons.list", {"leagueId": league_id}, f"seasons.list {league_id}")
    seasons = sresp["data"].get("seasons") or []
    return [s.get("strSeason") for s in seasons if s.get("strSeason")]

for li, L in enumerate(LEAGUES, 1):
    lid = L.get("idLeague"); lname = L.get("strLeague")
    if not lid:
        continue

    all_seasons = seasons_for_league(lid)
    if not all_seasons:
        continue
    chosen = all_seasons[-SEASONS_LIMIT:] if len(all_seasons) >= SEASONS_LIMIT else all_seasons

    for season in chosen:
        events_resp = call_ok("events.list", {"leagueId": lid, "season": season},
                              f"events.list {lid} {season}")
        events = events_resp["data"].get("events") or []
        events = events[:MAX_EVENTS_PER_SEASON]

        print(f"\n[LEAGUE {lname}] season {season} — scanning {len(events)} event(s)")
        for E in events:
            eid = E.get("idEvent")
            if not eid:
                continue

            # scan base row first
            base_urls = pick_video_fields(E)
            urls = set(base_urls)

            # pull details only if base had no links (to save calls)
            if not urls:
                try:
                    edetail = call_ok("event.get", {"eventId": eid, "expand": ["timeline","stats","lineup"]},
                                      f"event.get {eid}")
                    event_obj = edetail["data"].get("event") or {}
                    timeline = edetail["data"].get("timeline") or []
                    stats = edetail["data"].get("stats") or []
                    lineup = edetail["data"].get("lineup") or []

                    for u in pick_video_fields(event_obj):
                        urls.add(u)
                    for row in timeline:
                        for u in pick_video_fields(row):
                            urls.add(u)
                    for row in stats:
                        for u in pick_video_fields(row):
                            urls.add(u)
                    for row in lineup:
                        for u in pick_video_fields(row):
                            urls.add(u)
                except Exception as e:
                    print(f"[event.get WARN] {eid}: {e}")

            if urls:
                label = E.get("strEvent") or eid
                print(f"  [EVENT] {label} ({eid}) — {len(urls)} link(s)")
                for v in sorted(urls):
                    print("    •", v)

            time.sleep(SLEEP_BETWEEN)

    if li % 2 == 0:
        print(f"[events] leagues processed {li}/{len(LEAGUES)}")
    time.sleep(SLEEP_BETWEEN)

In [ ]:
# Cell 1 — helpers (run this first)
import time, requests, json
from typing import Dict, Any, List

BASE = "https://www.thesportsdb.com/api/v1/json/3"

def GET(path: str, params: Dict[str, Any]) -> Dict[str, Any]:
    """Minimal, robust GET with cache-buster and useful debug prints."""
    p = {**params, "_ts": f"{time.time():.6f}"}  # cache‑buster
    url = f"{BASE}/{path}"
    r = requests.get(url, params=p, timeout=20)
    ctype = r.headers.get("content-type", "")
    print(f"GET {url} params={params} -> {r.status_code} {ctype}")
    r.raise_for_status()
    # Guard: sometimes the service returns HTML (e.g., splash/rate-limit)
    if "json" not in ctype:
        preview = (r.text or "")[:120].replace("\n", " ")
        raise RuntimeError(f"Non-JSON body (status {r.status_code}, ctype {ctype}) preview='{preview}'")
    return r.json()

def print_kv(obj: Dict[str, Any], keys: List[str]):
    for k in keys:
        if k in obj and obj[k] not in (None, "", []):
            print(f"{k:>14}: {obj[k]}")

In [ ]:
# Cell 2 — TEAM sanity checks
from typing import Dict, Any
import pandas as pd

def team_by_name(team_name: str) -> Dict[str, Any]:
    t = (GET("searchteams.php", {"t": team_name}).get("teams") or [None])[0] or {}
    if not t:
        raise ValueError(f"No team found for name '{team_name}'")
    tid = t.get("idTeam")
    # Cross-check lookup by ID too
    t2 = (GET("lookupteam.php", {"id": tid}).get("teams") or [None])[0] or {}
    print(f"\n=== {team_name} → id={tid}")
    keep = ["idTeam","strTeam","strLeague","idLeague","strCountry","intFormedYear","strStadium","strWebsite","strYoutube"]
    print_kv(t2 or t, keep)
    return t2 or t

def teams_by_league_name(league_name: str):
    # Prefer name-based lookup; try with sport filter then fallback
    try:
        data = GET("search_all_teams.php", {"l": league_name, "s": "Soccer"})
    except Exception as e:
        print("[WARN] name+sport failed, retrying without sport:", e)
        data = GET("search_all_teams.php", {"l": league_name})
    rows = data.get("teams") or []
    print(f"\n=== Teams in '{league_name}' — {len(rows)} rows")
    # Print a compact list like your UI
    for r in rows[:30]:
        print("-", r.get("strTeam"), "| id", r.get("idTeam"))
    return rows

# EXAMPLE — change to whatever you’re testing:
_ = teams_by_league_name("English Premier League")
_ = teams_by_league_name("German Bundesliga")
_ = team_by_name("Arsenal")

In [ ]:
# Cell 3 — EVENTS sanity: list events for a given league *name* and season, pick an EVENT_ID
from typing import Optional, List

def resolve_league_id_by_name(league_name: str) -> str:
    data = GET("all_leagues.php", {})
    leagues = [L for L in (data.get("leagues") or []) if (L.get("strSport") or "").lower() == "soccer"]
    name_l = league_name.strip().lower()
    exact = [L for L in leagues if (L.get("strLeague") or "").strip().lower() == name_l]
    pick = exact[0] if exact else next((L for L in leagues if name_l in (L.get("strLeague") or "").lower()), None)
    if not pick:
        raise ValueError(f"League '{league_name}' not found in all_leagues")
    return str(pick["idLeague"])

def events_for_league_season(league_name: str, season: str, limit: Optional[int] = 10) -> List[Dict[str, Any]]:
    lid = resolve_league_id_by_name(league_name)
    evs = GET("eventsseason.php", {"id": lid, "s": season}).get("events") or []
    print(f"\n=== Events for {league_name} {season} — {len(evs)} returned")
    for e in evs[:limit or len(evs)]:
        print("-", e.get("dateEvent"), e.get("strEvent"), "| id=", e.get("idEvent"))
    return evs

# EXAMPLE — change these:
LEAGUE_NAME = "English Premier League"
SEASON = "2021-2022"

events = events_for_league_season(LEAGUE_NAME, SEASON, limit=12)
EVENT_ID = events[6]["idEvent"] if events else None
print("EVENT_ID =", EVENT_ID)

In [ ]:
# Cell 4 — EVENT details: fetch + compact summary (run after setting EVENT_ID in Cell 3)
from pprint import pprint
from typing import Dict, Any

def event_details(event_id: str, expand=("timeline","stats","lineup")) -> Dict[str, Any]:
    ev = (GET("lookupevent.php", {"id": event_id}).get("events") or [None])[0] or {}
    out = {"event": ev}
    if "timeline" in expand:
        out["timeline"] = GET("lookuptimeline.php", {"id": event_id}).get("timeline") or []
    if "stats" in expand:
        out["stats"] = GET("lookupeventstats.php", {"id": event_id}).get("eventstats") or []
    if "lineup" in expand:
        out["lineup"] = GET("lookuplineup.php", {"id": event_id}).get("lineup") or []
    return out

def print_event_summary(pkg: Dict[str, Any]):
    ev = pkg.get("event", {})
    print("\n=== Event Summary ===")
    print_kv(ev, ["idEvent","dateEvent","strEvent","strLeague","strHomeTeam","strAwayTeam","intHomeScore","intAwayScore","strStatus","strVenue","strVideo","strThumb"])
    tl = pkg.get("timeline", [])
    st = pkg.get("stats", [])
    lu = pkg.get("lineup", [])
    print("\nTimeline rows:", len(tl))
    if tl[:3]: pprint(tl[:3])
    print("\nStats rows:", len(st))
    if st[:3]: pprint(st[:3])
    print("\nLineup rows:", len(lu))
    if lu[:3]: pprint(lu[:3])

if 'EVENT_ID' not in globals() or not EVENT_ID:
    raise RuntimeError("Run Cell 3 first and set EVENT_ID from the printed list.")
pkg = event_details(EVENT_ID, expand=("timeline","stats","lineup"))
print_event_summary(pkg)

In [8]:
import requests, time

BASE = "https://www.thesportsdb.com/api/v1/json/3"
def GET(path, params=None):
    p = dict(params or {}); p["_ts"] = time.time()
    return requests.get(f"{BASE}/{path}", params=p, timeout=20).json()

# all soccer leagues
leagues = [L for L in GET("all_leagues.php").get("leagues", []) if L.get("strSport")=="Soccer"]
print("Total soccer leagues:", len(leagues))
for L in leagues[:34]:
    print(L["idLeague"], "-", L["strLeague"])

Total soccer leagues: 34
4328 - English Premier League
4329 - English League Championship
4330 - Scottish Premier League
4331 - German Bundesliga
4332 - Italian Serie A
4334 - French Ligue 1
4335 - Spanish La Liga
4336 - Greek Superleague Greece
4337 - Dutch Eredivisie
4338 - Belgian Pro League
4339 - Turkish Super Lig
4340 - Danish Superliga
4344 - Portuguese Primeira Liga
4346 - American Major League Soccer
4347 - Swedish Allsvenskan
4350 - Mexican Primera League
4351 - Brazilian Serie A
4354 - Ukrainian Premier League
4355 - Russian Football Premier League
4356 - Australian A-League
4358 - Norwegian Eliteserien
4359 - Chinese Super League
4367 - _No League
4394 - Italian Serie B
4395 - Scottish Championship
4396 - English League 1
4397 - English League 2
4398 - Italian Serie C Girone C
4399 - German 2. Bundesliga
4400 - Spanish La Liga 2
4401 - French Ligue 2
4403 - Swedish Superettan
4404 - Brazilian Serie B
4406 - Argentinian Primera Division


In [9]:
LEAGUE_ID = "4394"   # English Premier League (change as needed)

seasons = GET("search_all_seasons.php", {"id": LEAGUE_ID}).get("seasons") or []
print(f"League {LEAGUE_ID} has {len(seasons)} seasons")
for s in seasons[:10]:
    print("-", s["strSeason"])

League 4394 has 12 seasons
- 2012-2013
- 2013-2014
- 2016-2017
- 2017-2018
- 2018-2019
- 2019-2020
- 2020-2021
- 2021-2022
- 2022-2023
- 2023-2024


In [11]:
LEAGUE_ID = "4394"      # EPL
SEASON    = "2017-2018" # change as needed

events = GET("eventsseason.php", {"id": LEAGUE_ID, "s": SEASON}).get("events") or []
print(f"Matches returned: {len(events)}")
for e in events[:5]:
    print(e["idEvent"], e["dateEvent"], e["strEvent"], e["intHomeScore"], "-", e["intAwayScore"])

Matches returned: 100
529136 2017-08-25 Parma Calcio 1913 vs Cremonese 1 - 0
529137 2017-08-26 Ternana vs Empoli 1 - 1
529138 2017-08-26 Carpi vs Novara 1 - 0
529139 2017-08-26 Entella vs Perugia 1 - 5
529140 2017-08-26 Cittadella vs Ascoli 3 - 2


In [19]:
# Match details by event ID (ID-based)
EVENT_ID = "529137"   # example EPL match, change as needed

event = GET("lookupevent.php", {"id": EVENT_ID}).get("events", [None])[0]
print("Event ID lookup →", EVENT_ID)
for k, v in list(event.items())[:20]:   # show first 20 fields
    print(f"{k:20} {v}")

Event ID lookup → 529137
idEvent              441613
idAPIfootball        None
strEvent             Liverpool vs Swansea
strEventAlternate    Swansea @ Liverpool
strFilename          English Premier League 2014-12-29 Liverpool vs Swansea
strSport             Soccer
idLeague             4328
strLeague            English Premier League
strLeagueBadge       https://r2.thesportsdb.com/images/media/league/badge/dsnjpz1679951317.png
strSeason            2014-2015
strDescriptionEN     
strHomeTeam          Liverpool
strAwayTeam          Swansea
intHomeScore         4
intRound             19
intAwayScore         1
intSpectators        44621
strOfficial          None
strTimestamp         2014-12-29T20:00:00
dateEvent            2014-12-29


In [17]:
# Match details by event name (Name-based)
EVENT_NAME = "Parma Calcio 1913 vs Cremonese"   # change as needed

events = GET("searchevents.php", {"e": EVENT_NAME}).get("event") or []
print(f"Found {len(events)} match(es) for name='{EVENT_NAME}'")
if events:
    ev = events[0]
    for k, v in list(ev.items())[:20]:
        print(f"{k:20} {v}")

Found 3 match(es) for name='Parma Calcio 1913 vs Cremonese'
idEvent              1851829
idAPIfootball        1063331
strEvent             Parma Calcio 1913 vs Cremonese
strEventAlternate    Cremonese @ Parma Calcio 1913
strFilename          Italian Serie B 2024-05-05 Parma Calcio 1913 vs Cremonese
strSport             Soccer
idLeague             4394
strLeague            Italian Serie B
strLeagueBadge       https://r2.thesportsdb.com/images/media/league/badge/uf5kph1598011132.png
strSeason            2023-2024
strDescriptionEN     
strHomeTeam          Parma Calcio 1913
strAwayTeam          Cremonese
intHomeScore         1
intRound             37
intAwayScore         1
intSpectators        None
strOfficial          
strTimestamp         2024-05-05T13:00:00
dateEvent            2024-05-05


In [23]:
# Inputs you already have:
BASE = "https://www.thesportsdb.com/api/v1/json/3"
LEAGUE_ID = "4332"      # e.g., Italian Serie A
SEASON    = "2017-2018" # pick one

import requests, pandas as pd

def GET(ep, params):
    r = requests.get(f"{BASE}/{ep}", params=params, timeout=20)
    r.raise_for_status()
    return r.json()

# 1) Get all matches for league+season
season_events = GET("eventsseason.php", {"id": LEAGUE_ID, "s": SEASON}).get("events") or []
print("Matches returned:", len(season_events))

# 2) Choose the row you clicked in your UI (simulate with an index)
i = 2  # change this to the clicked index
chosen = season_events[i]
EVENT_ID   = chosen.get("idEvent")
EVENT_NAME = chosen.get("strEvent")
EVENT_DATE = chosen.get("dateEvent")
print(EVENT_ID, EVENT_DATE, EVENT_NAME)

# Keep a small snapshot for filtering later
chosen_min = {
    "idEvent": EVENT_ID,
    "strEvent": EVENT_NAME,
    "dateEvent": EVENT_DATE,
    "strSeason": chosen.get("strSeason"),
}
chosen_min

Matches returned: 100
528605 2017-08-20 Atalanta vs Roma


{'idEvent': '528605',
 'strEvent': 'Atalanta vs Roma',
 'dateEvent': '2017-08-20',
 'strSeason': '2017-2018'}

In [21]:
# Use the chosen_min from Cell 1
events_by_name = GET("searchevents.php", {"e": chosen_min["strEvent"]}).get("event") or []
print(f"Name search returned: {len(events_by_name)}")

# Filter priority: exact idEvent → same date+season → same date → first
candidates = events_by_name
exact_id = [e for e in candidates if e.get("idEvent") == chosen_min["idEvent"]]
if exact_id:
    ev = exact_id[0]
else:
    same_ds = [e for e in candidates
               if e.get("dateEvent") == chosen_min["dateEvent"]
               and e.get("strSeason") == chosen_min["strSeason"]]
    if same_ds:
        ev = same_ds[0]
    else:
        same_date = [e for e in candidates if e.get("dateEvent") == chosen_min["dateEvent"]]
        ev = (same_date or candidates or [{}])[0]

# Show a concise “details” view
key_order = ["idEvent","strEvent","strSeason","dateEvent","strTime","strLeague",
             "strHomeTeam","strAwayTeam","intHomeScore","intAwayScore",
             "strVenue","strCity","strCountry","strVideo","strThumb","strStatus"]
for k in key_order:
    if k in ev:
        print(f"{k:14} {ev.get(k)}")

Name search returned: 13
idEvent        528603
strEvent       Juventus vs Cagliari
strSeason      2017-2018
dateEvent      2017-08-19
strTime        16:00:00+00:00
strLeague      Italian Serie A
strHomeTeam    Juventus
strAwayTeam    Cagliari
intHomeScore   3
intAwayScore   0
strVenue       None
strCity        None
strCountry     None
strVideo       None
strThumb       None
strStatus      None
